In [ ]:
%pip install 'chronos-forecasting>=2.1' utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from chronos import BaseChronosPipeline, Chronos2Pipeline

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
# Initialize BaseChronosPipeline


In [ ]:
# Zero-shot forecasting


In [ ]:
fcsts_df = fcsts_df.drop(columns=["target_name"])
fcsts_df = fcsts_df.rename(columns={
    "target": "Weekly_Sales",
    "predictions": "Chronos2",
    "0.1": "Chronos2-lo-80",
    "0.9": "Chronos2-hi-80"
})

plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
# Cross-validation with Chronos-2
def chronos2_cv(
        df: pd.DataFrame,
        horizon: int,
        n_windows: int,
        id_col: str,
        time_col: str,
        target_col: str,
        futr_exog_cols: list[str] | None = None,
        hist_exog_cols: list[str] | None =None,
        step_size: int | None = None,
        cross_learning: bool = False
):
    # If step_size is None, use horizon for non-overlapping windows

    # List to have all results


    for i in range(n_windows):
        # Get test end offset

        # Create test set

        # Create input dataframe

        # Handle exogenous features columns

        # Prepare future_df

        # Make predictions

        # Rename columns
        preds_df = preds_df.rename(columns={
            "target": target_col,
            "predictions": "Chronos2",
            "0.1": "Chronos2-lo-80",
            "0.9": "Chronos2-hi-80"
        })

        # Merge actual values

        # Append results

    # Concatenate all results

    return all_preds_df

In [ ]:
# Run cross-validation

cv_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(columns=["Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    pd.DataFrame(cv_df),
    metrics=[mae, smape],
    models=['Chronos2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Forecasting with known future covariates

In [ ]:
futr_exog_cv_df = chronos2_cv(
    df,
    horizon=HORIZON,
    n_windows=3,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    # Specify known future covariates

)
futr_exog_cv_df.head()

In [ ]:
futr_exog_eval_df = evaluate(
    pd.DataFrame(futr_exog_cv_df),
    metrics=[mae, smape],
    models=['Chronos2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
futr_exog_eval_df

## Forecast with all exogenous features (future and historical)

In [ ]:
all_exog_cv_df = chronos2_cv(
    df,
    horizon=HORIZON,
    n_windows=3,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    futr_exog_cols=["Holiday_Flag"],
    # Specify historical features

)
all_exog_cv_df.head()

In [ ]:
all_exog_eval_df = evaluate(
    pd.DataFrame(all_exog_cv_df),
    metrics=[mae, smape],
    models=['Chronos2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
all_exog_eval_df

## Cross-learning (multivariate forecasting)

In [ ]:
mv_cv_df = chronos2_cv(
    df,
    horizon=HORIZON,
    n_windows=3,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    futr_exog_cols=["Holiday_Flag"],
    hist_exog_cols=["Temperature", "Fuel_Price", "CPI", "Unemployment"],
    # Use cross-learning for multivariate forecasting

)

mv_cv_df.head()

In [ ]:
mv_eval_df = evaluate(
    pd.DataFrame(mv_cv_df),
    metrics=[mae, smape],
    models=['Chronos2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
mv_eval_df

## Fine-tuning

In [ ]:
# Prepare data for fine-tuning
known_covariates = ["Holiday_Flag"]
past_covariates = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

# Specify training inputs


In [ ]:
# Fit the model (fine-tune it)


In [ ]:
def chronos2_cv(
        df: pd.DataFrame,
        horizon: int,
        n_windows: int,
        id_col: str,
        time_col: str,
        target_col: str,
        futr_exog_cols: list[str] | None = None,
        hist_exog_cols: list[str] | None = None,
        step_size: int | None = None,
        cross_learning: bool = False,
        # Add a parameter to pass a Chronos2Pipeline

):
    if step_size is None:
        step_size = horizon
    # If no pipeline is passed, create one


    # Rest of the function remains the same
    all_preds = []

    for i in range(n_windows):
        test_end_offset = horizon + (i * step_size)
        test = df.groupby(id_col).tail(test_end_offset)
        input_df = df.drop(test.index)

        input_cols = [id_col, time_col, target_col]
        if hist_exog_cols:
            input_cols += hist_exog_cols
        if futr_exog_cols:
            input_cols += futr_exog_cols
        input_df = input_df[input_cols]

        future_df = None
        if futr_exog_cols:
            future_df = test.groupby(id_col).head(horizon)[[id_col, time_col] + futr_exog_cols]

        preds_df = pipeline.predict_df(
            input_df,
            future_df=future_df,
            id_column=id_col,
            timestamp_column=time_col,
            target=target_col,
            prediction_length=horizon,
            quantile_levels=[0.1, 0.9],
            predict_batches_jointly=cross_learning,
        )

        preds_df = preds_df.rename(columns={
            "target": target_col,
            "predictions": "Chronos2",
            "0.1": "Chronos2-lo-80",
            "0.9": "Chronos2-hi-80"
        })
        preds_df = preds_df.merge(
            test[[id_col, time_col, target_col]],
            on=[id_col, time_col],
            how="left"
        )

        all_preds.append(preds_df)

    all_preds_df = pd.concat(all_preds)
    all_preds_df = all_preds_df.drop(columns=["target_name"])

    return all_preds_df

In [ ]:
ft_cv_df = chronos2_cv(
    df,
    horizon=HORIZON,
    n_windows=3,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    futr_exog_cols=["Holiday_Flag"],
    hist_exog_cols=["Temperature", "Fuel_Price", "CPI", "Unemployment"],
    cross_learning=True,
    # Specify the finetuned pipeline

)

ft_cv_df.head()

In [ ]:
ft_eval_df = evaluate(
    pd.DataFrame(ft_cv_df),
    metrics=[mae, smape],
    models=['Chronos2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
ft_eval_df